In [ ]:
#| default_exp 04_download-wikipedia-dataset

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import requests, os, re, argparse, multiprocessing as mp, ssl
from timeit import default_timer as timer
from functools import partial
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
#| export
def download_file(url, cache_dir):
    fn = f'{cache_dir}/{os.path.basename(url)}'
    urlretrieve(url, filename=fn)

In [ ]:
#| export
def _full_condition(o):
    if 'pages-articles-multistream' in o and '.xml-p' not in o and '.txt-p' not in o: return True
    else: return False

def _parts_condition(o):
    if 'pages-articles-multistream' in o and ('.xml-p' in o or '.txt-p' in o): return True
    else: return False

def _abstract_condition(o):
    if '-abstract.xml.gz' in o: return True
    else: return False

def get_condition(dtype):
    if dtype == 'full': 
        return _full_condition
    elif dtype == 'parts': 
        return _parts_condition
    elif dtype == 'abstract': 
        return _abstract_condition
    else: 
        raise ValueError(f'Invalid condition type: {dtype}')

In [ ]:
#| export
def download_wikipedia_dataset(data_url, cache_dir, condition_type):
    os.makedirs(cache_dir, exist_ok=True)

    html = requests.get(data_url).text
    soup_handler = BeautifulSoup(html, 'html.parser')

    all_files = [file.text.split(' ')[0] for file in soup_handler.find_all('li', {'class': 'file'})]
    condition = get_condition(condition_type)
    urls = [f'{data_url}/{o}' for o in all_files if condition(o)]
    print(f'Total number of file: {len(urls)}')

    for url in tqdm(urls): download_file(url, cache_dir)

In [ ]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_url', type=str, default='https://dumps.wikimedia.org/')
    parser.add_argument('--condition_type', type=str, default='full')
    parser.add_argument('--project', type=str, required=True)
    parser.add_argument('--date', type=str, required=True)
    parser.add_argument('--cache_dir', type=str, required=True)
    return parser.parse_args()

if __name__ == '__main__':
    start_time = timer()

    args = parse_args()

    data_url = f'{args.data_url}/{args.project}/{args.date}'
    download_wikipedia_dataset(data_url, args.cache_dir, args.condition_type)

    end_time = timer()
    print(f'Time elapsed: {end_time-start_time:.2f} seconds.')